## Импорты

In [2]:
import pandas as pd

In [11]:
from pyvis.network import Network

## Загрузка данных

Данные взяты с сайта https://drugcentral.org/

Данные в виде Postgres базы данных, из которой вручную достаются нужные данные в агрегированном виде

In [156]:
drugs[drugs['class_id'] == 7737.0]

,name,class_id
9,abacavir,7737.0
29,acefylline,7737.0
107,agalsidase alfa,7737.0
115,alacepril,7737.0
155,alizapride,7737.0
...,...,...
5207,thiocolchicoside,7737.0
5231,tiadenol,7737.0
5279,tinofedrine,7737.0
5409,triclofos,7737.0


In [157]:
counts = drugs['name'].value_counts()
class_ids_with_more_than_3_counts = counts[counts > 3].index.tolist()
result = drugs[drugs['name'].isin(class_ids_with_more_than_3_counts)]
result

,name,class_id
39,acetaminosalol,7754.0
40,acetaminosalol,5387.0
41,acetaminosalol,7720.0
42,acetaminosalol,7712.0
43,acetaminosalol,7747.0
...,...,...
5660,zimeldine,7710.0
5661,zimeldine,7711.0
5662,zimeldine,7736.0
5663,zimeldine,7709.0


In [4]:
drugs = pd.read_csv('./drugs_groups.csv')
drugs.head()

,name,class_id
0,(2-benzhydryloxyethyl)diethyl-methylammonium i...,5323.0
1,(S)-nicardipine,NaN
2,(S)-nitrendipine,5454.0
3,1-Octacosanol,7362.0
4,2-(4-chlorphenoxy)-ethanol,NaN


In [5]:
groups = pd.read_csv('./drugs_groups_compability.csv')
groups.head() 

,class_name1,class_id1,class_name2,class_id2,ddi_risk,description
0,Monoamine Oxidase Inhibitors,7740,Alpha/Beta Agonists,7741,Potentially significant,MAO Inhibitors may enhance the hypertensive ef...
1,Monoamine Oxidase Inhibitors,7740,Amphetamines,7742,Potentially significant,MAO Inhibitors may enhance the hypertensive ef...
2,Alpha/Beta Agonists,7741,Inhalational Anesthetics,7743,Potentially significant,Inhalational Anesthetics may enhance the arrhy...
3,Monoamine Oxidase Inhibitors,7740,buspirone,6131,Potentially significant,May increase blood pressure
4,Monoamine Oxidase Inhibitors,7740,dextromethorphan,4933,Contraindicated,fatal/non-fatal reactions


In [10]:
print(groups.ddi_risk.unique())

['Potentially significant' 'Contraindicated' 'Avoid combination'
 'Significant' 'Critical']


## Создание графовой модели

In [182]:
main_drug = 'maltose' # 'acetaminosalol' - большой граф

In [183]:
net = Network(
    notebook=True,
    height="750px", 
    width="100%", 
    bgcolor="#222222", 
    font_color="white"
)
net.add_node(main_drug, color='#d9ce34', size=10)

In [184]:
main_classes = drugs[drugs.name == main_drug]
main_classes_list = list(main_classes.class_id)

related_classes_set = set()

if not(classes_id == []):
    for r in groups[groups.class_id1.isin(main_classes_list)].iterrows():
        # добавляем класс центрального препарата
        net.add_node(r[1][1], label=r[1][0], color='#9e0909', size=15)
                
        # добавляем связь препарата с классом
        net.add_edge(main_drug, r[1][1], color='#f5f74f', width=3)
        
        # добавляем связанный класс
        related_classes_set.add(r[1][3])
        net.add_node(r[1][3], label=r[1][2], color='#d94343', size=15)
        
        # добавляем связь между классами
        net.add_edge(r[1][1], r[1][3], label=r[1][5], color='#12a4de', width=5)
        
    for r in groups[groups.class_id2.isin(main_classes_list)].iterrows():
        # добавляем класс центрального препарата
        net.add_node(r[1][3], label=r[1][2], color='#9e0909', size=15)
                
        # добавляем связь препарата с классом
        net.add_edge(main_drug, r[1][3], color='#f5f74f', width=3)
        
        # добавляем связанный класс
        related_classes_set.add(r[1][1])
        net.add_node(r[1][1], label=r[1][0], color='#d94343', size=15)
        
        # добавляем связь между классами
        net.add_edge(r[1][1], r[1][3], label=r[1][5], color='#12a4de', width=5)
        
    for cls in related_classes_set:
        for drug in list(drugs[drugs.class_id == cls].name):
            # добавляем ноду препарата
            net.add_node(drug, color='#0aa123', size=10)

            # добавляем связь препарата с классом
            net.add_edge(drug, cls, color='#3cf06f', width=2)

In [185]:
# ## рисует полный граф, полный граф очень лагает


# for index, row in groups.iterrows():
    
#     net.add_node(row['class_id1'], label=row['class_name1'], color='#FF4136', size=30)
#     net.add_node(row['class_id2'], label=row['class_name2'], color='#FF4136', size=30)
    
#     net.add_edge(row['class_id1'], row['class_id2'], label=row['ddi_risk'], color='#0074D9', width=5)


# for index, row in drugs.iterrows():
#     if row['class_id'] is not None:
        
#         if row['class_id'] in groups['class_id1'].tolist() or row['class_id'] in groups['class_id2'].tolist():
            
#             net.add_node(index, label=row['name'], color='#2ECC40', size=15)
#             net.add_edge(row['class_id'], index, color='#2ECC40', width=1)
#         else:
            
#             net.add_node(index, label=row['name'], color='#2ECC40', size=15)



In [186]:
# net.toggle_physics(False) # рисует быстрее, но граф не очень красивый
net.set_options("""
const options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -10000
    },
    "minVelocity": 0.75
  }
}
""")
net.show('my_graph.html')

my_graph.html
